In [11]:
import os
import sys
import pickle
import numpy as np
from numpy import gradient
from math import pi
from packages import data_container
from packages.data_container import Data
from packages.helper import traj_speed, beta, min_dist, d_psi, min_sep
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

In [12]:
'''Import data from csv to Data object'''
header=('subj_x', 'subj_y', 'subj_z', 'subj_yaw', 'subj_pitch', 'subj_row')
info={'p_goal':[], 'p_obst':[], 'obst_angle':[], 'obst_speed':[], 'subj_id':[], 'trial_id':[], 'stimuli_onset':[], 
      'stimuli_out':[], 'w_goal': 0.4, 'w_obst': 0.2, 'ps_trial':[], 'ps_subj':[], 'time_stamp':[], 'stimuli_match':[],
      'pass_order':[], 'p_subj':[], 'v_subj':[], 'v_obst':[], 'a_subj':[]}
data = Data(Hz=90, header=header, info=info)
n_subj = 11
for subj_id in range(n_subj):
    # Import output
    file_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1',
                                             'Subj' + str(subj_id).zfill(2) + '.csv'))
    with open(file_path, 'r') as f:
        trials = [line.split(',') for line in f.read().split('\n')]
    head = 0
    while head + 1 < len(trials):
        for i in range(head+1, len(trials)):
            if trials[i][0] == '':
                tail = i - 1
                break
        trial_id = int(trials[head+1][1])
        time_stamp = [float(line[0]) for line in trials[head+3:tail+1]]
        angle = float(trials[head+1][3])
        speed = float(trials[head+1][5])
        subj = np.array([[float(val) for val in line[1:7]] for line in trials[head+3:tail+1]])
        p0 = Data.filter(subj[:,[0,2]], time_stamp, 90, 4, 0.6)
        v0 = gradient(p0, axis=0) * data.Hz
        a0 = gradient(v0, axis=0) * data.Hz
        goal = np.array([[float(line[-2]), float(line[-1])] for line in trials[head+3:tail+1]])
        obst = np.array([[float(line[-4]), float(line[-3])] for line in trials[head+3:tail+1]])
        p_obst = Data.filter(obst, time_stamp, data.Hz, 4, 0.6)
        if speed:
            for i in range(len(obst) - 1):
                if obst[i][0] != obst[i + 1][0] or obst[i][1] != obst[i + 1][1]:
                    onset = i
                    break
            for i in range(onset, len(subj) - 1):
                _beta = beta(p0[i], p_obst[i], v0[i])
                if abs(_beta) > pi / 2:
                    stimuli_out = i
                    # 1: pass in front of obst, -1: pass behind obst, 0: freewalk trial
                    pass_order = np.sign(_beta * -angle)
                    break
            v1 = (obst[-1] - obst[onset]) / (len(obst) - 1 - onset) * data.Hz
            stimuli_match = onset
            for i in range(onset, stimuli_out):
                # The first moment when pass_order == order_pred
#                     order_pred = np.sign(min_sep(p0[i], obst[i], v0[i], v1)[0])
                order_pred = np.sign(beta(p0[i], p_obst[i], v0[i]) * d_psi(p0[i], p_obst[i], v0[i], v1))
                if pass_order * order_pred > 0:
                    stimuli_match = i
                    break
        else:
            pass_order = 0  # 1: pass in front of obst, -1: pass behind obst, 0: freewalk trial
            onset = stimuli_out = stimuli_match = None
        data.add_traj(subj)
        ps_trial = np.mean(traj_speed(subj[-2*data.Hz:-data.Hz,[0,2]], data.Hz))
        data.add_info({'p_goal':goal, 'p_obst':p_obst, 'obst_angle':angle, 'obst_speed':speed, 'subj_id':subj_id, 
                       'trial_id':trial_id, 'stimuli_onset':onset, 'stimuli_out':stimuli_out, 'time_stamp':time_stamp, 
                       'ps_trial':ps_trial, 'ps_subj':0, 'pass_order':pass_order, 'stimuli_match':stimuli_match,
                       'p_subj':p0, 'v_subj':v0, 'a_subj':a0, 'v_obst':v1})
        head = tail + 1
ps_subj = [0] * n_subj
for i in range(len(data.trajs)):
    if data.info['obst_speed'][i] == 0:
        subj_id = data.info['subj_id'][i]
        ps_subj[subj_id] += data.info['ps_trial'][i] / 10
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    data.info['ps_subj'][i] = ps_subj[subj_id]
outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1_data.pickle'))       
with open(outfile, 'wb') as f:   
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)  

In [15]:
data.info['stimuli_out']

[707,
 619,
 578,
 654,
 638,
 562,
 608,
 None,
 626,
 608,
 654,
 588,
 520,
 665,
 637,
 668,
 592,
 596,
 617,
 640,
 623,
 642,
 None,
 654,
 564,
 634,
 600,
 635,
 632,
 579,
 632,
 669,
 568,
 634,
 None,
 634,
 580,
 599,
 576,
 642,
 616,
 593,
 585,
 631,
 623,
 665,
 666,
 620,
 651,
 588,
 622,
 615,
 615,
 630,
 655,
 619,
 645,
 615,
 641,
 None,
 617,
 609,
 631,
 652,
 625,
 533,
 639,
 657,
 653,
 606,
 614,
 617,
 None,
 615,
 627,
 665,
 551,
 623,
 618,
 600,
 654,
 627,
 603,
 624,
 656,
 571,
 644,
 605,
 614,
 642,
 661,
 627,
 None,
 609,
 660,
 668,
 631,
 618,
 650,
 667,
 625,
 632,
 554,
 602,
 608,
 585,
 579,
 598,
 656,
 None,
 610,
 637,
 608,
 665,
 629,
 613,
 608,
 605,
 None,
 617,
 635,
 535,
 669,
 636,
 605,
 617,
 563,
 636,
 618,
 624,
 541,
 605,
 555,
 None,
 654,
 670,
 623,
 605,
 635,
 564,
 623,
 669,
 612,
 654,
 666,
 609,
 576,
 656,
 620,
 653,
 555,
 650,
 594,
 635,
 585,
 645,
 629,
 636,
 592,
 None,
 509,
 None,
 580,
 478,
 515,

In [10]:
'''Check different start time'''
for i in range(100):
    t0 = data.info['stimuli_onset'][i]
    t0_match = data.info['stimuli_match'][i]
    t1 = data.info['stimuli_out'][i]
    if t0 != t0_match:
        print(t0, t0_match, t1)

240 313 654
233 391 638
236 246 608
256 264 667
245 312 590
234 374 617
249 268 622
238 293 635
253 394 632
235 243 669
232 291 621
244 465 651
247 254 641
239 252 615
245 246 639
245 338 653
241 353 606
228 366 617
237 238 602
259 297 656
243 440 644
248 297 661
245 251 626
238 245 668
268 270 631
245 303 667


In [11]:
'''Check matching rate'''
for i in range(1400):
    if not data.info['obst_speed'][i]:
        continue
    pass_order = data.info['pass_order'][i]
    t0 = data.info['stimuli_onset'][i]
    t0_match = data.info['stimuli_match'][i]    
    p0 = data.info['p_subj'][i][t0_match]
    v0 = data.info['v_subj'][i][t0_match]
    p1 = data.info['p_obst'][i][t0_match]
    v1 = data.info['v_obst'][i]
    order_pred = np.sign(beta(p0, p1, v0) * d_psi(p0, p1, v0, v1))
    if pass_order * order_pred < 0:
        print('subj ', data.info['subj_id'][i], 'trial ', data.info['trial_id'][i], 
              'angle ', data.info['obst_angle'][i],
              pass_order, order_pred, beta(p0, p1, v0) * d_psi(p0, p1, v0, v1))

subj  0 trial  36 angle  180.0 -1.0 1.0 4.560314074852409e-06
subj  0 trial  46 angle  180.0 -1.0 1.0 5.5418204009773305e-05
subj  0 trial  47 angle  180.0 -1.0 1.0 0.00016201213168408237
subj  0 trial  48 angle  180.0 -1.0 1.0 9.370492103510116e-05
subj  0 trial  71 angle  180.0 -1.0 1.0 3.700563095644856e-05
subj  0 trial  84 angle  180.0 -1.0 1.0 0.00010697799813246213
subj  0 trial  88 angle  180.0 -1.0 1.0 0.00022204153090218026
subj  0 trial  89 angle  180.0 -1.0 1.0 0.00012127946579508302
subj  0 trial  114 angle  -180.0 -1.0 1.0 1.4428094998683392e-06
subj  0 trial  128 angle  180.0 -1.0 1.0 1.2225563898930016e-05
subj  0 trial  141 angle  180.0 -1.0 1.0 2.3192131891055343e-05
subj  1 trial  21 angle  180.0 -1.0 1.0 0.001007009349672666
subj  1 trial  23 angle  180.0 -1.0 1.0 0.00015070386191967338
subj  1 trial  39 angle  -180.0 -1.0 1.0 0.0004894877986861775
subj  1 trial  40 angle  180.0 -1.0 1.0 3.261316583271917e-05
subj  1 trial  87 angle  180.0 -1.0 1.0 0.000714905572195